In [1]:
#<imports>
import os
from getpass import getpass

from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.prompts.prompt import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import HuggingFaceHub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
#<\imports>

In [2]:
#<get hf token>
HF_TOKEN = getpass("Huggingface Token : ")
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_TOKEN
#<\get hf token>

In [3]:
#<load docs and split to chunks>
loader = PyPDFDirectoryLoader('data/')
documents = loader.load()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)
#<\load docs and split to chunks>

In [4]:
#<prompt template>
prompt_template = """
<|system|>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
Your answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
#<\prompt template>

In [ ]:
#<init embeddings and vec store>
embeddings = HuggingFaceEmbeddings(model_name="thenlper/gte-large")
vectorstore = Chroma.from_documents(chunks, embeddings)

query = "what is electrolysis"
search = vectorstore.similarity_search(query)
#<\init embeddings and vec store>

In [6]:
#<create bm25 and vector retrievers>
bm25_retriever = BM25Retriever.from_documents(chunks)
vector_retriever = vectorstore.as_retriever()
#\<create bm25 and vector retrievers>

In [7]:
#<set up ensemble retriever>
retrievers = [bm25_retriever, vector_retriever]
ensemble_retriever = EnsembleRetriever(retrievers=retrievers, weights=[0.5, 0.5])
#<\set up ensemble retriever>

In [ ]:
#<init llm>
llm = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-beta",
                     task="text-generation",
                     model_kwargs={
                         "max_new_tokens": 512,
                         "top_k": 30,
                         "temperature": 0.1,
                         "repetition_penalty": 1.1,
                         "return_full_text":False
                         },
                    )
#<\init llm>

In [9]:
#<make rag pipe>
output_parser = StrOutputParser()

retriever= ensemble_retriever
chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)
#<\make rag pipe>

In [ ]:
#<test query>
query = "what is an electrolyte"
response = chain.invoke(query)
print(response)

print(chain.invoke("what is electrolysis?"))

print(chain.invoke("why do we dream?"))
#<\test query>